In [1]:
import os
import cv2
import keras 
import numpy as np
import pandas as pd

def train_test_split(rd,train_ratio):

    fls = np.asarray(os.listdir(rd)) #Get Files
    np.random.shuffle(fls) #Get IID Array s

    train_length = int(np.ceil(train_ratio * (len(fls))))
    test_length = len(fls) - train_length

    train = fls[0:train_length] 
    test = fls[-test_length:]

    return train, test 

def load(rd,file_names):

    X = np.zeros(shape = (len(file_names),150,150,3),dtype=np.float16)
    Y = np.zeros(shape=(len(file_names)), dtype=object)

    for idx,sub_dir in enumerate(file_names):

        if('.DS' in sub_dir):
            continue

        dr = os.path.join(rd,sub_dir)
        im = cv2.imread(os.path.join(dr,"gsv_0.jpg")) 
        im = im / 255

        X[idx,:,:,:] = im
        Y[idx] = sub_dir.split(" ")[0]

        # meta = os.path.join(dr,'metadata.json')
        # if(json.load(open(meta))[0]['status'] == 'OK'): 
            
    return X, Y

2023-05-23 17:13:25.093486: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/gpoulsen@apamail.org/Library/Python/3.10/lib/python/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
trn, tst = train_test_split('/Users/gpoulsen@apamail.org/Desktop/Grant Training/Geoguesser/extracted_data',.8) #Break Directories into a train and test set
X_train,Y_train = load("/Users/gpoulsen@apamail.org/Desktop/Grant Training/Geoguesser/extracted_data",trn) #Load the data set
X_test,Y_test = load("/Users/gpoulsen@apamail.org/Desktop/Grant Training/Geoguesser/extracted_data",tst) #Load the data set

In [3]:
from keras.models import Sequential
import keras.layers as kl

model =  Sequential()

model.add(kl.Conv2D(input_shape=(150,150,3), filters=64, kernel_size=(3,3), activation='relu', padding='same')) #Input 
model.add(kl.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')) #Convolution 1

model.add(kl.MaxPool2D(pool_size=(2,2),strides=(2,2))) #Pool 2

model.add(kl.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')) #Convolution 3
model.add(kl.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')) #Convolution 4

model.add(kl.MaxPool2D(pool_size=(2,2),strides=(2,2))) #Pool 5

model.add(kl.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')) #Convolution 6
model.add(kl.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')) #Convolution 7
model.add(kl.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')) #Convolution 8

model.add(kl.MaxPool2D(pool_size=(2,2),strides=(2,2))) #Pool 9

model.add(kl.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same')) #Convolution 10
model.add(kl.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same')) #Convolution 11
model.add(kl.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same')) #Convolution 12

model.add(kl.MaxPool2D(pool_size=(2,2),strides=(2,2))) #Pool 13

model.add(kl.Dense(units=4096)) #Fully Connected 14
model.add(kl.Dense(units=4096)) #Fully Connected 15

model.add(kl.Dense(units=4, activation='softmax')) #Softmax Output  

from keras.optimizers import Adam

opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

#create callback
filepath = '/Users/gpoulsen@apamail.org/Desktop/IGP EOY/my_best_model.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
callbacks = [checkpoint]

EPOCHS = 10

In [ ]:
EPOCHS = 10
checkpoint_filepath = '/tmp/checkpoint'

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(

    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Model weights are saved at the end of every epoch, if it's the best seen
# so far.
model.fit(epochs=EPOCHS, callbacks=[model_checkpoint_callback])

# The model weights (that are considered the best) are loaded into the
# model.
model.load_weights(checkpoint_filepath)

In [16]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=100,generator=traindata, validation_data= testdata, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 150, 150, 64)      1792      
                                                                 
 conv2d_21 (Conv2D)          (None, 150, 150, 64)      36928     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 75, 75, 64)       0         
 2D)                                                             
                                                                 
 conv2d_22 (Conv2D)          (None, 75, 75, 128)       73856     
                                                                 
 conv2d_23 (Conv2D)          (None, 75, 75, 128)       147584    
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 37, 37, 128)      0         
 2D)                                                  